In [16]:
# from google.colab import drive
# drive.mount('/content/drive')
!pip install tensorflow

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import wrangle as w 
pd.set_option('display.max_columns', None)

In [3]:
# Load the dataset
df = w.prepare_third_filtered_dataset_version()

In [4]:
cols_to_drop = ['crash_id', 'person_injury_severity', 'injury_binary']

In [5]:
df_encoded = df.copy()

for col in df.columns:
    if col not in cols_to_drop:
        df_encoded = pd.get_dummies(df_encoded, columns=[col], drop_first=True, dtype=int)

x= df_encoded.drop(columns=cols_to_drop)

In [6]:
# Split the data into training, validation, and test sets
train, validate, test = w.split(df_encoded)

In [7]:
X_train = train.drop(columns=cols_to_drop)
y_train = train['injury_binary']

X_validate = validate.drop(columns=cols_to_drop)
y_validate = validate['injury_binary']

X_test = test.drop(columns=cols_to_drop)
y_test = test['injury_binary']

In [8]:
# START HERE 

In [17]:
# Import necessary libraries for ANN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Define the model - shallow neural net
model = Sequential()

# Add model layers
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[Precision()])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_validate, y_validate), epochs=100, callbacks=[early_stopping])

NameError: name 'Precision' is not defined

In [10]:
# The model was trained for 6 epochs out of a total of 100. 
# The training loss decreased over these epochs, indicating that the model was learning from the training data. 
# However, the validation loss increased after the second epoch, suggesting that the model might be overfitting to the training data. 
# Overfitting occurs when a model learns the training data too well, including its noise and outliers, and performs poorly on new, unseen data.

# The accuracy of the model on both the training and validation data remained relatively stable, around 91-92%.
# This could mean that the model has reached its capacity with the current architecture and parameters, and further training might not improve the performance significantly.

In [14]:
# Evaluate the model on the test set
test_loss, test_precision = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Precision: {test_precision}')

# Evaluate the model on the validation set
val_loss, val_precision = model.evaluate(X_validate, y_validate)
print(f'Validation Loss: {val_loss}')
print(f'Validation Precision: {val_precision}')


89/89 [==============================] - 0s 416us/step - loss: 0.3896 - accuracy: 0.9098
Test Loss: 0.38961583375930786
Test Precision: 0.9097983837127686
107/107 [==============================] - 0s 376us/step - loss: 0.3265 - accuracy: 0.9251
Validation Loss: 0.32651183009147644
Validation Precision: 0.9251400232315063


In [12]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_validate, y_validate)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

107/107 [==============================] - 0s 395us/step - loss: 0.3265 - accuracy: 0.9251
Validation Loss: 0.32651183009147644
Validation Accuracy: 0.9251400232315063


In [13]:
# Predict the probabilities of the classes on the test set
y_pred_proba = model.predict(x)
print(y_pred_proba)

442/442 [==============================] - 0s 347us/step
[[0.99340415]
 [0.8989543 ]
 [0.9440586 ]
 ...
 [0.9979201 ]
 [0.99809647]
 [0.9272422 ]]
